In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle, islice

In [ ]:
data = '../data/'
data_out = '../results/data_out/'
output_folder = '../results/data_out/results_sensitivity/annual_change_sens/'
fig_folder = '../results/figs/'
version = '3_oct_2022'
ylim = (-250000, 700000)
prefix = '4_'

In [ ]:
scens = ['95% by 2035', 'Reference']
global_scen = '95% by 2035'
ind_agg = pd.read_csv('../results/data_out/Data_out_names/ind_sum_aggr.csv', index_col=0)['label aggr']
order_flat_to_bell = False

In [ ]:
short_ind_names = {'Agriculture, Forestry, Fishing and Hunting': 'Agriculture',
 'Mining, Quarrying, and Oil and Gas Extraction': 'Mining (incl. fossil fuels)',
 'Utilities': 'Utilities',
 'Construction': 'Construction',
 'Manufacturing': 'Manufacturing',
 'Wholesale Trade': 'Wholesale',
 'Retail Trade': 'Retail',
 'Transportation and Warehousing': 'Transportation',
 'Information': 'Information',
 'Finance and Insurance': 'Finance',
 'Real Estate and Rental and Leasing': 'Real Estate',
 'Professional, Scientific, and Technical Services': 'Professional Services',
 'Management of Companies and Enterprises': 'Company Management',
 'Administrative and Support and Waste Management and Remediation Services': 'Administrative Services',
 'Educational Services': 'Education',
 'Health Care and Social Assistance': 'Health Care',
 'Arts, Entertainment, and Recreation': 'Arts',
 'Accommodation and Food Services': 'Accommodation and Food',
 'Other Services (except Public Administration)': 'Other Services',
 'Public Administration': 'Public Administration'}

In [ ]:
ind_agg = ind_agg.map(short_ind_names)

In [ ]:
for global_scen in scens:
    print(global_scen)
    df_full = pd.DataFrame()
    for yr in range(2021, 2050, 1):
        df = pd.read_csv(output_folder + prefix + 'ind_empl_' + \
                                            global_scen + '_' + str(yr-1) + '-'+ str(yr) + \
                                             '.csv', index_col=0)
        df['year'] = yr
        df_full = pd.concat([df_full, df])

    print(df_full.sum().sum())
    df_full = df_full.reset_index().set_index(['index', 'year'])
    df_try = df_full.unstack().T.copy()
    df_try.index = df_try.index.droplevel(0)
    df_try = df_try.groupby(df_try.index).sum()
    df_try.loc[2020, :] = 0
    df_try.sort_index(inplace=True)
    
    
    utils = set(df_try) - set(ind_agg.index)

    utils = dict(zip(utils, ['Utilities' for x in utils]))

    ind_agg = pd.concat([ind_agg, pd.Series(utils, dtype=str)])

    indcat = np.unique(ind_agg)
    my_colors = list(islice(cycle(['lime',  'violet', 'y', 'darkred', 'aquamarine', 
                                   'b', 'r', 'pink', 'peru', 'k', 'grey',
                                   'slategrey', 'midnightblue', 'salmon', 'goldenrod', 
                                   'teal', 'purple', 
                                   'deeppink', 'g', 'm', 'c']), None, 
                            len(indcat)))
    colordict = dict(zip(indcat, my_colors))

#     my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k', 'm', 'c', 'lime', 
#                                    'slategrey', 'midnightblue', 'salmon', 'goldenrod', 
#                                    'teal', 'purple', 'violet', 'pink', 'peru', 'grey']), None, 2*len(df_try.columns)))

    
    to_plot = pd.concat([df_try.cumsum().clip(lower=0), df_try.cumsum().clip(upper=0)], axis=1)

    if order_flat_to_bell == True:
        to_order = df_try.cumsum().copy()
        idx_order = np.abs(to_order.loc[2033] / to_order.loc[2037]).fillna(0).sort_values().index
        if global_scen == '95% by 2050':
            idx_order = idx_order[::-1]
        to_plot = to_plot.loc[:,idx_order]
    

    colorlist = [colordict[ind_agg.to_dict()[ind]] for ind in to_plot.columns]
        
    to_plot.plot.area(color=colorlist, figsize=(6,6), ylim=ylim, legend='reverse')#.\
                    #legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        
    markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colordict.values()]
    plt.legend(markers, colordict.keys(), numpoints=1, loc='center left', bbox_to_anchor=(-0.06, -1.1), ncols=2)#, bbox_to_anchor=(1.0, 0.5))

    sns.despine()

    plt.ylabel('Cumulative worker demand \nin the electricity supply chain') 
    plt.tight_layout()
    plt.show()

In [ ]:
major_occ = pd.read_csv('../results/data_out/results_sensitivity/annual_change_sens/4_empl_95% by 2035_2021-2022.csv', index_col=0)['OCC_TITLE_major']

In [ ]:
short_major_occ_names = {'Management Occupations': 'Management',
 'Business and Financial Operations Occupations': 'Business & Financial',
 'Computer and Mathematical Occupations': 'Computer & Maths',
 'Architecture and Engineering Occupations': 'Engineering',
 'Life, Physical, and Social Science Occupations': 'Science Occupations',
 'Community and Social Service Occupations': 'Social Service',
 'Legal Occupations': 'Legal',
 'Education, Training, and Library Occupations': 'Education',
 'Arts, Design, Entertainment, Sports, and Media Occupations': 'Arts',
 'Healthcare Practitioners and Technical Occupations': 'Healthcare Pract.',
 'Healthcare Support Occupations': 'Healthcare Support',
 'Protective Service Occupations': 'Protective Service',
 'Food Preparation and Serving Related Occupations': 'Food Preparation',
 'Building and Grounds Cleaning and Maintenance Occupations': 'Cleaning and Maintenance ',
 'Personal Care and Service Occupations': 'Personal Care',
 'Sales and Related Occupations': 'Sales',
 'Office and Administrative Support Occupations': 'Administrative',
 'Farming, Fishing, and Forestry Occupations': 'Farming',
 'Construction and Extraction Occupations': 'Construction & Extraction',
 'Installation, Maintenance, and Repair Occupations': 'Installation & Repair',
 'Production Occupations': 'Production',
 'Transportation and Material Moving Occupations': 'Transportation'}

In [ ]:
cols = ['Wind-capex', 'Solar-capex',
       'Natural gas-capex', 'Coal-capex', 'Biomass-capex', 'Geothermal-capex',
       'Hydro-capex', 'Battery storage-capex', 'T&D-capex', 'Biomass-opex',
       'Coal-opex', 'Solar-opex', 'Natural gas-opex', 'Geothermal-opex',
       'Hydro-opex', 'Nuclear-opex', 'Wind-opex',
       'Transmission and distribution-opex', 'Battery storage-opex']

#for global_scen in scens:
global_scen = scens[0]
print(global_scen)
df_full = pd.DataFrame()
for yr in range(2021, 2050, 1):
    df = pd.read_csv(output_folder + prefix + 'empl_' + \
                                        global_scen + '_' + str(yr-1) + '-'+ str(yr) + \
                                         '.csv', index_col=0)
    df['year'] = yr
    df_full = pd.concat([df_full, df])

df_full = df_full.reset_index().set_index(['index', 'year'])
df_try = df_full.loc[:,cols].unstack().T.copy()
df_try.index = df_try.index.droplevel(0)
df_try = df_try.groupby(df_try.index).sum()
df_try.loc[2020, :] = 0
df_try.sort_index(inplace=True)

short_majocc = major_occ.map(short_major_occ_names)

occcat = np.unique(short_majocc)

my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k', 'm', 'c', 'lime', 
                               'slategrey', 'midnightblue', 'salmon', 'goldenrod', 
                               'teal', 'purple', 'violet', 'pink', 'peru', 'grey', 
                               'deeppink', 'olive', 'darkred', 'aquamarine']), None, 
                        len(occcat)))
colordict = dict(zip(occcat, my_colors))
colorlist = [colordict[occ] for occ in short_majocc]


pd.concat([df_try.cumsum().clip(lower=0), df_try.cumsum().clip(upper=0)], axis=1).plot.area(color=colorlist, figsize=(6,6), ylim=ylim, legend='reverse')#.\
                #legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colordict.values()]

plt.legend(markers, colordict.keys(), numpoints=1, loc='center left', bbox_to_anchor=(0, -0.7), ncols=2)

sns.despine()

plt.ylabel('Cumulative new jobs in electricity supply chain')     
plt.show()

In [ ]:
cols = ['Wind-capex', 'Solar-capex',
       'Natural gas-capex', 'Coal-capex', 'Biomass-capex', 'Geothermal-capex',
       'Hydro-capex', 'Battery storage-capex', 'T&D-capex', 'Biomass-opex', 'Coal-opex',
       'Solar-opex', 'Natural gas-opex', 'Geothermal-opex', 'Hydro-opex',
       'Nuclear-opex', 'Wind-opex', 'Transmission and distribution-opex',
       'Battery storage-opex']

#for global_scen in scens:
global_scen = scens[1]
print(global_scen)
df_full = pd.DataFrame()
for yr in range(2021, 2050, 1):
    df = pd.read_csv(output_folder + prefix + 'empl_' + \
                                        global_scen + '_' + str(yr-1) + '-'+ str(yr) + \
                                         '.csv', index_col=0)
    df['year'] = yr
    df_full = pd.concat([df_full, df])

df_full = df_full.reset_index().set_index(['index', 'year'])
df_try = df_full.loc[:,cols].unstack().T.copy()
df_try.index = df_try.index.droplevel(0)
df_try = df_try.groupby(df_try.index).sum()
df_try.loc[2020, :] = 0
df_try.sort_index(inplace=True)

occcat = np.unique(major_occ.str[:-12])
my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k', 'm', 'c', 'lime', 
                               'slategrey', 'midnightblue', 'salmon', 'goldenrod', 
                               'teal', 'purple', 'violet', 'pink', 'peru', 'grey', 
                               'deeppink', 'olive', 'darkred', 'aquamarine']), None, 
                        len(occcat)))
colordict = dict(zip(occcat, my_colors))
colorlist = [colordict[occ] for occ in major_occ.str[:-12]]


pd.concat([df_try.cumsum().clip(lower=0), df_try.cumsum().clip(upper=0)], axis=1).plot.area(color=colorlist, figsize=(6,6), ylim=ylim, legend='reverse')#.\
                #legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colordict.values()]
plt.legend(markers, colordict.keys(), numpoints=1, loc='center left', bbox_to_anchor=(1.0, 0.5))

plt.ylabel('Cumulative new jobs in electricity supply chain')     
plt.show()